In [1]:
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import cv2
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, precision_score, recall_score, matthews_corrcoef
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from torchvision import transforms
from torchvision.transforms import v2
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss
from torch.utils import data
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [8]:
OR_PATH = os.getcwd()
PATH = OR_PATH + os.path.sep + "Deborah" + os.path.sep + "Final-Project-Group4"

FILENAME = PATH + os.path.sep + "dataset" + os.path.sep + "final_dataset.xlsx"
DATA_DIR = PATH + os.path.sep + "dataset" + os.path.sep + "train" + os.path.sep
ATTRIBUTES_DIR = PATH + os.path.sep + "dataset" + os.path.sep + "attributes.xlsx"

#Inception net must have image sizes as 299 * 299
n_epoch = 5
BATCH_SIZE = 32
LR = 0.0005

## Image processing
CHANNELS = 3
IMAGE_SIZE = 224

NICKNAME = "Group4"

mlb = MultiLabelBinarizer()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
THRESHOLD = 0.5
SAVE_MODEL = True

In [3]:
attr_data = pd.read_excel(ATTRIBUTES_DIR)
attributes = attr_data['name']


#main data
xdf_data = pd.read_excel(FILENAME)

xdf_dtrain = xdf_data[xdf_data['Split'] == 'train'].copy()
xdf_dtest = xdf_data[xdf_data['Split'] == 'test'].copy()


In [6]:
#Function to process attributes features
def preprocess_attributes(attributes):
    attributes = attributes.to_list()
    num_attribute_classes = len(attribute_map)
    attribute_vectors = []
    
    for attr_string in attributes:
        attribute_vector = torch.zeros(num_attribute_classes)
        for attr_index, attr_names in attribute_map.items():
            if attr_string in attr_names.split(', '):
                attribute_vector[attr_index] = 1
        attribute_vectors.append(attribute_vector)
    
    return attribute_vectors


attribute_map = attributes.to_dict()
preprocess_attributes(attributes)[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:

""" Custom data class to read in the images one sample at a time using the image ImageId

    Processing the multilabel target(catgeory) into one-hot-encoding
    
    Returns the transformed images and labels

"""

class ImageDataset(data.Dataset):
    def __init__(self, list_IDs, type_data, target_type):
        #Initialization'
        self.type_data = type_data
        self.list_IDs = list_IDs
        self.target_type = target_type
        
        self.transforms = v2.Compose([
            transforms.Resize((224, 224)),
            v2.CenterCrop(224),
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0),
            transforms.RandomGrayscale([0.2]),
            v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        
    #Denotes the total number of samples'  
    def __len__(self):
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        
        #Genrate one sample at a time by using loading the image file and the id
        ID = self.list_IDs[index]

        # Load data and get label

        if self.type_data == 'train':
            y = xdf_dtrain.target_class.get(ID)
            if self.target_type == 2:
                y = y.split(",")
        else:
            y = xdf_dtest.target_class.get(ID)
            if self.target_type == 2:
                y = y.split(",")


        if self.target_type == 2:
            labels_ohe = [ int(e) for e in y]
        else:
            labels_ohe = np.zeros(OUTPUTS_a)

            for idx, label in enumerate(range(OUTPUTS_a)):
                if label == y:
                    labels_ohe[idx] = 1

        y = torch.FloatTensor(labels_ohe)

        if self.type_data == 'train':
            file = DATA_DIR + xdf_dtrain.ImageId.get(ID)
        else:
            file = DATA_DIR + xdf_dtest.ImageId.get(ID)

        img = cv2.imread(file)

        img= cv2.resize(img,(IMAGE_SIZE, IMAGE_SIZE))

        # Augmentation for training
        X = torch.FloatTensor(img)

        X = torch.reshape(X, (3, IMAGE_SIZE, IMAGE_SIZE))

        if self.type_data == 'train':
            X = self.transforms(X)


        return X, y
    





def process_target(target_type):
    '''
        1- Binary   target = (1,0)
        2- Multiclass  target = (1...n, text1...textn)
        3- Multilabel target = ( list(Text1, Text2, Text3 ) for each observation, separated by commas )
    :return:
    '''

    dict_target = {}
    xerror = 0

    if target_type == 2:
        ## The target comes as a string  x1, x2, x3,x4
        ## the following code creates a list
        target = np.array(xdf_data['Category'].apply( lambda x : x.split(",")))
        final_target = mlb.fit_transform(target)
        xfinal = []
        if len(final_target) ==0:
            xerror = 'Could not process Multilabel'
        else:
            class_names = mlb.classes_
            for i in range(len(final_target)):
                joined_string = ",".join( str(e) for e in final_target[i])
                xfinal.append(joined_string)
            xdf_data['target_class'] = xfinal

    if target_type == 1:
        xtarget = list(np.array(xdf_data['Category'].unique()))
        le = LabelEncoder()
        le.fit(xtarget)
        final_target = le.transform(np.array(xdf_data['Category']))
        class_names=(xtarget)
        xdf_data['target_class'] = final_target
        
        
    ## Calculate class weights based on the frequency of each class in the dataset, finally worked!!!
    label_matrix = xdf_dtrain['Category'].str.get_dummies(",")
    class_dist = label_matrix.sum()
    total_num_samples = class_dist.sum()
    class_weights = total_num_samples / (class_dist * len(class_dist))
    class_weight_tensor = torch.tensor(class_weights.values, dtype=torch.float)
    sample_weights = label_matrix.dot(class_weight_tensor).values

    return class_names, sample_weights
    

class_names, sample_weights = process_target(2)
OUTPUTS_a = len(class_names)


xdf_dtrain = xdf_data[xdf_data['Split'] == 'train'].copy()
xdf_dtest = xdf_data[xdf_data['Split'] == 'test'].copy()  
